# Term Project
- Student Name: Christohper Poon
- Student UT EID: cmp2263


- Partner Name: Michael Thrash
- Partner UT EID: mat5429

# Research Question: 
- 1st Question: We want to classify a pokemon image and predict the what the pokemon's primary typing will be of the 18 different types
    - We Expect the results to be better than 1/18 or 0.05 but not very high due to background pixel noise
    
    - for simplicity we only looked at primary typing and not dual typing of the pokemon
    
- 2nd Question: We want to classify an image and predict if the pokemon's image is a water type or not a water type
    - We expect the results to be much higher since the classification only includes two different labels 
    
- Classification will be evaulated on the Accuracy Score of corretly getting the pokemon's type

In [73]:
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics

In [134]:
import requests
from io import BytesIO
for i in range(1,906):
    stop = i
    url = 'https://bulbapedia.bulbagarden.net/wiki/File:HOME{0:03}.png'.format(i)    

    response = requests.get(url)
    start = str(response.content).index("content=\"https")
    end = str(response.content)[start:].index(".png")+4
    url = str(response.content)[start+9:start+end]
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img.save(str(i)+".png")

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x00000283A96B9D50>

# Convert RGBA values to integers by processing them as base 32 integers and only keeping those that have alpha>16

In [74]:
def rgba_to_int(tup):
    if(tup[3]<16):
        return 0
    f1 = 32.0
    f2 = 256.0/f1
    return f2*(f2*float(round(tup[0]/f1)) + float(round(tup[1]/f1))) + float(round(tup[2]/f1))
vect_rgba_to_int = np.vectorize(rgba_to_int)

# Get color frequencies for images of pokemon

In [75]:
def get_pixel_list(col_dict, dex_number):
    im = Image.open('C:\\Users\\micha\\Downloads\\pokemon_images\\'+str(dex_number+1)+'.png')
    im = im.convert(mode = 'RGBA')
    val = np.zeros(int((256/32)**3))
    col_series = pd.Series(val)
    pixels = list(im.getdata())
    pixels = pd.Series(pixels).apply(rgba_to_int).value_counts()
#    pixels = pd.Series(pixels.values,index = pd.Index(vect_rgba_to_int(pixels.index)))
    col_series.update(pixels)
    col_dict[dex_number] = col_series
#    print(dex_number)
gpl_vect = np.vectorize(get_pixel_list)

# Make the dataframe for the color frequencies 

In [105]:
color_dict = dict()

gpl_vect(color_dict,np.arange(0,905))
pokemon_df = pd.DataFrame(color_dict).T
nametypes = pd.read_csv('pokemon.csv')

In [106]:
# Get rid of Nan values and convert type 2 to type 1 for all single types
index_df = nametypes.loc[nametypes['Type2'].isna()].index
nametypes['Type2'].iloc[index_df] = nametypes['Type1'].iloc[index_df]

In [107]:
s_typings = pd.Series(nametypes['Type1'].unique())
list_types = s_typings.values
list_num_types = np.arange(len(list_types))
dict_typing = dict(zip(list_types,list_num_types))
num_typing = dict(zip(list_num_types, list_types))
dict_typing
num_typing

{0: 'Grass',
 1: 'Fire',
 2: 'Water',
 3: 'Bug',
 4: 'Normal',
 5: 'Poison',
 6: 'Electric',
 7: 'Ground',
 8: 'Fairy',
 9: 'Fighting',
 10: 'Psychic',
 11: 'Rock',
 12: 'Ghost',
 13: 'Ice',
 14: 'Dragon',
 15: 'Dark',
 16: 'Steel',
 17: 'Flying'}

In [108]:
def change_type_to_num(string_type):
    return dict_typing[string_type]

In [109]:
def num_to_type(num):
    return num_typing[num]

# y1 label: Type 1 all 18 types

In [110]:
y1 = pd.Series(nametypes['Type1'])
y1 = y1.apply(change_type_to_num)
y1

0       0
1       0
2       0
3       1
4       1
       ..
900     7
901     2
902     9
903    15
904     8
Name: Type1, Length: 905, dtype: int32

In [111]:
y2 = pd.Series(nametypes['Type2'])
y2 = y2.apply(change_type_to_num)
y2

0       5
1       5
2       5
3       1
4       1
       ..
900     4
901    12
902     5
903     5
904    17
Name: Type2, Length: 905, dtype: int32

In [112]:
df_X = pokemon_df.drop(columns = 0)

In [113]:
df_X

,1,2,3,4,5,6,7,8,9,10,...,502,503,504,505,506,507,508,509,510,511
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,652.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,...,39.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,861.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,39.0,...,202.0,269.0,1.0,1.0,2.0,1.0,1.0,0.0,18.0,1678.0
3,6.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,43.0,6.0,...,97.0,0.0,0.0,0.0,4.0,600.0,3149.0,18.0,198.0,410.0
4,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,...,510.0,5.0,0.0,0.0,2.0,5.0,251.0,3085.0,53.0,1393.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,71.0,0.0,0.0,0.0,0.0,20.0,0.0,2.0,18.0,3244.0
901,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,411.0,263.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,2071.0
902,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,52.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2072.0
903,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,9.0,...,313.0,150.0,11.0,0.0,0.0,0.0,0.0,250.0,7281.0,509.0


# All 18 Labels Random Forest Classifier

In [117]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 1024)
#model = AdaBoostClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_test,y_pred))
precise_accuracy = np.sum(y_pred == y_test) / len(y_pred)
#adjacent_accuracy = np.sum(np.abs(y_pred - y_test) <= 1) / len(y_pred)
print(precise_accuracy)
#print(adjacent_accuracy)
print(classification_report(y_test,y_pred))

0.430939226519337
0.430939226519337
              precision    recall  f1-score   support

           0       0.41      0.86      0.56        14
           1       0.86      0.75      0.80        16
           2       0.50      0.59      0.54        27
           3       0.30      0.21      0.25        14
           4       0.28      0.71      0.41        21
           5       0.25      0.33      0.29         6
           6       0.50      0.42      0.45        12
           7       1.00      0.10      0.18        10
           8       0.00      0.00      0.00         3
           9       0.00      0.00      0.00         5
          10       0.40      0.50      0.44         8
          11       1.00      0.15      0.27        13
          12       1.00      0.20      0.33         5
          13       0.50      0.75      0.60         4
          14       0.00      0.00      0.00         7
          15       0.50      0.33      0.40         6
          16       0.00      0.00      0.00  

C:\Users\micha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\micha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\micha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

# References:
https://archives.bulbagarden.net/wiki/Category:HOME_artwork